In [1]:
# Import necessary libraries
import numpy as np
import random


### تنظیمات اولیه
در این بخش، پارامترهای اولیه الگوریتم Q-Learning تعریف می‌شوند.


In [2]:
# تنظیمات اولیه
gamma = 0.9  # ضریب تخفیف
alpha = 0.1  # نرخ یادگیری
epsilon = 0.1  # احتمال انتخاب عمل تصادفی (Exploration vs Exploitation)

# تعریف محیط (جدول 5x5)
n_states = 25  # تعداد کل حالات (5x5)
n_actions = 4  # تعداد کل اعمال (بالا، پایین، چپ، راست)
goal_state = 24  # هدف در آخرین خانه
actions = ['up', 'down', 'left', 'right']

### تعریف Q-Table
در این قسمت، جدول Q برای نگهداری مقادیر Q برای هر حالت و عمل تعریف می‌شود.


In [3]:
# تعریف Q-Table
Q = np.zeros((n_states, n_actions))


### تابعی برای محاسبه حالت بعدی و پاداش
این تابع، با توجه به وضعیت فعلی و عملی که انتخاب می‌شود، وضعیت بعدی و پاداش آن را محاسبه می‌کند.


In [4]:
# تابعی برای محاسبه حالت بعدی و پاداش
def step(state, action):
    row, col = divmod(state, 5)  # تبدیل ایندکس به ردیف و ستون
    if action == 0 and row > 0:  # حرکت به بالا
        row -= 1
    elif action == 1 and row < 4:  # حرکت به پایین
        row += 1
    elif action == 2 and col > 0:  # حرکت به چپ
        col -= 1
    elif action == 3 and col < 4:  # حرکت به راست
        col += 1
    next_state = row * 5 + col  # تبدیل دوباره ردیف و ستون به ایندکس
    reward = 1 if next_state == goal_state else -0.01  # پاداش 1 اگر هدف رسید، در غیر این صورت -0.01
    return next_state, reward


### اجرای الگوریتم SARSA
در این بخش، الگوریتم SARSA برای آموزش عامل بر روی محیط تعریف شده اجرا می‌شود.


In [5]:
# الگوریتم SARSA
episodes = 1000  # تعداد اپیزودها برای آموزش
for episode in range(episodes):
    state = random.randint(0, n_states - 1)  # شروع از یک حالت تصادفی
    
    # انتخاب عمل اولیه
    if random.uniform(0, 1) < epsilon:
        action = random.randint(0, n_actions - 1)  # انتخاب عمل تصادفی (جستجو)
    else:
        action = np.argmax(Q[state])  # انتخاب بهترین عمل (استفاده از اطلاعات قبلی)

    while state != goal_state:
        # اجرای عمل و دریافت پاداش
        next_state, reward = step(state, action)
        
        # انتخاب عمل بعدی (a') برای حالت بعدی (s')
        if random.uniform(0, 1) < epsilon:
            next_action = random.randint(0, n_actions - 1)  # انتخاب عمل تصادفی (جستجو)
        else:
            next_action = np.argmax(Q[next_state])  # انتخاب بهترین عمل (استفاده از اطلاعات قبلی)
        
        # به‌روزرسانی Q-table با استفاده از الگوریتم SARSA
        Q[state, action] = Q[state, action] + alpha * (
            reward + gamma * Q[next_state, next_action] - Q[state, action]
        )
        
        # انتقال به حالت و عمل بعدی
        state = next_state
        action = next_action


### نمایش Q-Table
در اینجا جدول Q که عامل طی آموزش آن را یاد گرفته است، نمایش داده می‌شود.


In [6]:
# نمایش Q-Table
print("Q-Table:")
print(Q)


Q-Table:
[[-0.0126868  -0.00936192 -0.01347368  0.20390928]
 [ 0.06788093  0.29082534 -0.01083267 -0.01026732]
 [-0.00741958  0.4037036   0.02812211  0.06008175]
 [ 0.03308344 -0.00563537 -0.00593044  0.47750081]
 [ 0.09466373  0.60457671  0.08156098  0.12108797]
 [-0.01214427  0.25283546 -0.01187582  0.02166681]
 [ 0.0628309   0.01989408  0.01303053  0.40365199]
 [ 0.10234776  0.52952136  0.10213352  0.12831645]
 [-0.0014911   0.13532236  0.17589967  0.62673772]
 [ 0.15094995  0.75837134  0.14075186  0.35028727]
 [ 0.0120356   0.06525298  0.02634345  0.4481053 ]
 [ 0.05331236  0.4810969   0.0817965   0.06626379]
 [ 0.18544333  0.64459586  0.05200946  0.27248506]
 [ 0.09527286  0.70441382  0.16677818  0.07242323]
 [ 0.15577662  0.85256079  0.11627755  0.20160926]
 [ 0.030588    0.0217068  -0.0065549   0.48593445]
 [ 0.18558352  0.57406294  0.07894695  0.07895927]
 [ 0.26816244  0.42162406  0.28083853  0.73885924]
 [ 0.23391532  0.50603093  0.38022571  0.82456505]
 [ 0.55805566  1.     

### آزمایش عامل آموزش‌دیده
در این قسمت، عامل آموزش‌دیده به شروع از خانه اول می‌پردازد و مسیر خود را تا رسیدن به هدف طی می‌کند.


In [7]:
# آزمایش عامل آموزش‌دیده
state = 0  # شروع از اولین حالت (گوشه بالا-چپ)
path = [state]
while state != goal_state:
    action = np.argmax(Q[state])  # انتخاب بهترین عمل
    state, _ = step(state, action)  # انتقال به حالت بعدی
    path.append(state)

print("\nمسیر به سمت هدف:", path)



مسیر به سمت هدف: [0, 1, 6, 7, 12, 17, 18, 19, 24]
